#### Imports

In [23]:
import numpy as np
import scipy
import sympy as sp
from IPython.display import display

import matplotlib.pyplot as plt

#### Deriving discrete dynamics

In [ ]:
A_continuous = sp.Matrix([[0, 1], [0, 0]])
B_continuous = sp.Matrix([[0], [1]])
dt = sp.symbols('dt')
x1, x2 = sp.symbols('x1 x2')
x = sp.Matrix([[x1], [x2]])
u = sp.symbols('u')

k1 = A_continuous * x + B_continuous * u
k2 = A_continuous * (x + 0.5 * dt * k1) + B_continuous * u
k3 = A_continuous * (x + 0.5 * dt * k2) + B_continuous * u
k4 = A_continuous * (x + dt * k3) + B_continuous * u

x_new = sp.simplify(x + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4))
display(x_new)

A_discrete = sp.Matrix([[1, dt], [0, 1]])
B_discrete = sp.Matrix([[dt**2 / 2], [dt]])
print("A_discrete")
display(A_discrete)
print("B_discrete")
display(B_discrete)
display(A_discrete * x + B_discrete * u)
display(A_discrete * x + B_discrete * u - x_new)

#### Compute Sigma_K

In [ ]:
def x0_distribution():
    return np.random.multivariate_normal(np.zeros(2), np.eye(2))

def get_sigma_k_exact(A,B,K):
    closed_loop = A - B @ K
    return scipy.linalg.solve_discrete_lyapunov(closed_loop, np.eye(2))

def get_sigma_k_mc(A,B,K,x0_distribution,num_samples=1000,max_timesteps=50):
    total = np.zeros((2,2))
    for i in range(num_samples):
        x0 = x0_distribution()
        tt = A - B @ K
        for j in range(max_timesteps):
            total += np.outer(x0, x0)
            x0 = tt @ x0
    return total / num_samples

dt = 0.1
A = np.array([[1, dt], [0, 1]])
B = np.array([[dt**2 / 2], [dt]])
K = np.array([[0.5, 0.3]])

print("Exact")
print(get_sigma_k_exact(A,B,K))

sanity_check = True
if sanity_check:    
    print("MC")
    print(get_sigma_k_mc(A,B,K,x0_distribution, num_samples=3000, max_timesteps=200))


#### Compute P_K and CK

In [26]:
def get_P_K(K, A, B, Q, R):
    return scipy.linalg.solve_discrete_lyapunov((A - B @ K).T, Q + K.T @ R @ K)

def get_cost(K, A, B, Q, R):
    return np.trace(get_P_K(K, A, B, Q, R))

#### Discrete Double Integrator

In [27]:
dt = 0.1
A = np.array([[1, dt], [0, 1]])
B = np.array([[dt**2 / 2], [dt]])
Q = np.eye(2) * 2
R = np.array([[1]])

#### Check gradient domination

In [28]:
def check_gradient_domination(K, K_opt, A, B, Q, R, grad_C_K):    
    Ck = get_cost(K, A, B, Q, R)
    Ck_opt = get_cost(K_opt, A, B, Q, R)
    Sigma_K = get_sigma_k_exact(A,B,K)
    Sigma_K_opt = get_sigma_k_exact(A,B,K_opt)

    # Get minimum singular value of Sigma_K_opt
    min_sv_K = np.min(np.linalg.svd(Sigma_K)[1])
    min_sv_R = np.min(np.linalg.svd(R)[1])

    C_diff = Ck - Ck_opt
    ub = np.linalg.norm(Sigma_K_opt, ord=2)
    ub = ub / (min_sv_K**2 * min_sv_R) * np.linalg.norm(grad_C_K)**2
    assert ub > C_diff, f"C_diff_upper: {ub}, C_diff: {C_diff}"
        

#### Exact Policy Gradient Algorithms

In [ ]:
def compute_optimal_K(A,B,Q,R,alpha, K0 = np.array([[1, 1]]), max_iterations=1000):
    
    K = K0
    
    for i in range(max_iterations):
        P_K = get_P_K(K, A, B, Q, R)

        # Compute Sigma_K
        Sigma_K = get_sigma_k_exact(A,B,K)
        
        # Compute E_K
        E_K = (R + B.T @ P_K @ B) @ K - B.T @ P_K @ A
        
        # Compute gradient
        grad_C_K = 2 * E_K @ Sigma_K
        
        # Gauss-Newton update
        K_new = K - alpha * scipy.linalg.inv(R + B.T @ P_K @ B) @ grad_C_K @ scipy.linalg.inv(Sigma_K)
        
        K = K_new
    
    return K

K_opt = compute_optimal_K(A,B,Q,R,alpha=0.02, max_iterations=5000)
print(f"Optimal K: {K_opt}")

In [ ]:
def exact_policy_gradient(A,B,Q,R,alpha, K0 = np.array([[1,1]]), max_iterations=1000):
    
    K = K0
    distances = []
    
    for i in range(max_iterations):
        P_K = get_P_K(K, A, B, Q, R)

        # Compute Sigma_K
        Sigma_K = get_sigma_k_exact(A,B,K)
        
        # Compute E_K
        E_K = (R + B.T @ P_K @ B) @ K - B.T @ P_K @ A
        
        # Compute gradient
        grad_C_K = 2 * E_K @ Sigma_K

        if np.linalg.norm(grad_C_K) > 1e-4:
            check_gradient_domination(K, K_opt, A, B, Q, R, grad_C_K)
        
        # Gauss-Newton update
        K_new = K - alpha * grad_C_K
        
        # Store distance to optimal K
        distances.append(np.linalg.norm(K - K_opt))
        
        K = K_new
    
    return K, distances

K, distances_pg = exact_policy_gradient(A,B,Q,R,alpha=0.02, max_iterations=3000)

# Plot distances over iterations
plt.figure(figsize=(10,6))
plt.plot(distances_pg)
plt.xlabel('Iteration')
plt.ylabel('Distance to Optimal K')
plt.title('Convergence of Policy Gradient')
plt.grid(True)
plt.show()

print(f"Optimal K: {K_opt}")
print(f"Computed K: {K}")


In [ ]:
def exact_natural_policy_gradient(A,B,Q,R,alpha, K0 = np.array([[1, 1]]), max_iterations=1000):
    
    K = K0
    distances = []
    
    for i in range(max_iterations):
        # Compute P_K
        P_K = get_P_K(K, A, B, Q, R)
        
        # Compute Sigma_K
        Sigma_K = get_sigma_k_exact(A,B,K)
        
        # Compute E_K
        E_K = (R + B.T @ P_K @ B) @ K - B.T @ P_K @ A
        
        # Compute gradient
        grad_C_K = 2 * E_K @ Sigma_K

        if np.linalg.norm(grad_C_K) > 1e-4:
            check_gradient_domination(K, K_opt, A, B, Q, R, grad_C_K)
        
        # Gauss-Newton update
        K_new = K - alpha * grad_C_K @ scipy.linalg.inv(Sigma_K)

        # Store distance to optimal K
        distances.append(np.linalg.norm(K - K_opt))
        
        K = K_new
    
    return K, distances

K, distances_ng = exact_natural_policy_gradient(A,B,Q,R,alpha=0.02, max_iterations=3000)

# Plot distances over iterations
plt.figure(figsize=(10,6))
plt.plot(distances_ng)
plt.xlabel('Iteration')
plt.ylabel('Distance to Optimal K')
plt.title('Convergence of Natural Policy Gradient')
plt.grid(True)
plt.show()

print(f"Optimal K: {K_opt}")
print(f"Computed K: {K}")

In [ ]:
def exact_gauss_newton(A,B,Q,R,alpha, K0 = np.array([[1, 1]]), max_iterations=1000):
    
    K = K0
    distances = []
    
    for i in range(max_iterations):
        # Compute P_K
        P_K = get_P_K(K, A, B, Q, R)
        
        # Compute Sigma_K
        Sigma_K = get_sigma_k_exact(A,B,K)
        
        # Compute E_K
        E_K = (R + B.T @ P_K @ B) @ K - B.T @ P_K @ A
        
        # Compute gradient
        grad_C_K = 2 * E_K @ Sigma_K

        if np.linalg.norm(grad_C_K) > 1e-4:
            check_gradient_domination(K, K_opt, A, B, Q, R, grad_C_K)
        
        # Gauss-Newton update
        K_new = K - alpha * scipy.linalg.inv(R + B.T @ P_K @ B) @ grad_C_K @ scipy.linalg.inv(Sigma_K)
        
        # Store distance to optimal K
        distances.append(np.linalg.norm(K - K_opt))
        
        K = K_new
    
    return K, distances

K, distances_gn = exact_gauss_newton(A,B,Q,R,alpha=0.02, max_iterations=3000)

# Plot distances over iterations
plt.figure(figsize=(10,6))
plt.plot(distances_gn)
plt.xlabel('Iteration')
plt.ylabel('Distance to Optimal K')
plt.title('Convergence of Gauss-Newton Method')
plt.grid(True)
plt.show()

print(f"Optimal K: {K_opt}")
print(f"Computed K: {K}")

#### Plot performance

In [ ]:
N = 300
font_size = 17
line_width = 3
plt.figure(figsize=(10,6))
plt.plot(distances_pg[:N], label="Policy Gradient", linewidth=line_width)
plt.plot(distances_ng[:N], label="Natural Policy Gradient", linewidth=line_width) 
plt.plot(distances_gn[:N], label="Gauss-Newton", linewidth=line_width)
plt.xlabel('Iteration', fontsize=font_size)
plt.ylabel('Distance to Optimal K', fontsize=font_size)
plt.title('Convergence of Different Methods', fontsize=font_size)
plt.grid(True)
plt.grid(True, linestyle='--', alpha=0.7, which='both')
plt.minorticks_on()
plt.grid(True, which='minor', linestyle=':', alpha=0.4)
plt.legend(fontsize=font_size)
plt.savefig("convergence.png", dpi=300)
plt.show()


In [ ]:
# Load distances from file
with open("distances_pg.txt", "r") as file:
    distances_pg = [float(line.strip()) for line in file]

with open("distances_npg.txt", "r") as file:
    distances_npg = [float(line.strip()) for line in file]
# Compare distances
N = 3000
font_size = 17
line_width = 3
plt.figure(figsize=(10,6))
plt.plot(distances_pg[:N], label="Approximate Policy Gradient", linewidth=line_width)
plt.plot(distances_npg[:N], label="Approximate Natural Policy Gradient", linewidth=line_width)
plt.xlabel('Iteration', fontsize=font_size)
plt.ylabel('Distance to Optimal K', fontsize=font_size)
plt.title('Convergence of Different Methods', fontsize=font_size)
plt.grid(True)
plt.grid(True, linestyle='--', alpha=0.7, which='both')
plt.minorticks_on()
plt.grid(True, which='minor', linestyle=':', alpha=0.4)
plt.legend(fontsize=font_size)
plt.savefig("convergence.png", dpi=300)
plt.show()




#### Approximate Policy Gradient Algorithms

In [ ]:
import numpy as np

# Function to sample initial state from a standard normal distribution
def sample_initial_state(d):
    return np.random.normal(0, 1, size=(d,))

# Function to compute Frobenius norm of a matrix
def frobenius_norm(mat):
    return np.sqrt(np.sum(np.square(mat)))

def policy_gradient_estimation_naive(K, m, roll_out_length, r, d, A, B, Q, R):
    C_estimates = []
    sigma_estimates = []

    for _ in range(m):
        # Sample random matrix U_i with Frobenius norm r
        U_i = np.random.uniform(-1, 1, size=K.shape)
        U_i *= r / frobenius_norm(U_i)

        # Perturbed policy K_i = K + U_i
        K_i = K + U_i

        # Initialize arrays for trajectory
        total_cost = 0.0
        states = []

        # Sample initial state
        x = sample_initial_state(d)

        for _ in range(roll_out_length):
            states.append(x)
            u = -np.dot(K_i, x)
            total_cost += np.dot(x.T, np.dot(Q, x)) + np.dot(u.T, np.dot(R, u))
            x = np.dot(A, x) + np.dot(B, u)

        # Calculate empirical estimates
        C_i = total_cost
        Sigma_i = np.zeros((d, d))
        for s in states:
            Sigma_i += np.outer(s, s)

        C_estimates.append((C_i, U_i))
        sigma_estimates.append(Sigma_i)

    # Calculate final estimates
    gradient_estimate = np.zeros(K.shape)
    for C_i, U_i in C_estimates:
        gradient_estimate += (d / (m * r * r)) * C_i * U_i

    sigma_estimate = np.mean(sigma_estimates, axis=0)

    return gradient_estimate, sigma_estimate

def model_free_natural_policy_gradient(A, B, Q, R, K0, m, roll_out_length, r, max_iterations=1000, eta=0.01):
    K = K0
    d = A.shape[0]
    
    for _ in range(max_iterations):
        # Get empirical estimates
        grad_estimate, sigma_estimate = policy_gradient_estimation_naive(K, m, roll_out_length, r, d, A, B, Q, R)
        
        print("Gradient estimate:\n", grad_estimate)
        
        # Natural policy gradient update
        K_new = K - eta * grad_estimate
        
        # Check convergence
        if np.linalg.norm(K - K_new) < 1e-8:
            break
        
        K = K_new
        print("Current K:\n", K)
    
    return K

d = 2
m = 3000
roll_out_length = 150
r = 0.001

dt = 0.1
K = np.array([[1, 1]])
A = np.array([[1, dt], [0, 1]])
B = np.array([[0.5 * dt ** 2], [dt]])
Q = np.eye(d) * 2
R = np.eye(1)

K0 = np.array([[1.0, 1.0]])

print("Starting model-free natural policy gradient optimization...")

Kest = model_free_natural_policy_gradient(
    A, B, Q, R, K0, 
    m=3000,
    roll_out_length=200,
    r=0.1,
    max_iterations=10000,
    eta=0.0005
)

print("Final K:\n", Kest)